In [1]:
import pandas as pd
import numpy as np

# 可視化用のライブラリ
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import neologdn
import MeCab

import re

from tqdm import tqdm_notebook as tqdm
from tqdm._tqdm_notebook import tqdm_notebook
# # プログレスバーのラベル設定
tqdm_notebook.pandas(desc="progress: ")

In [2]:
# 予めtf値とidf値を紐付けていたcsvファイルを読み込む
idf_merge_df = pd.read_csv('idf_merge_df.csv', encoding='shift_jis')
idf_merge_df.head()

,内容,original_type,count,word_count,tf,word_include_text_count,all_text_count,idf
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,(_記号-一般,1,26,0.038462,1,4,2.386294
1,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,)_記号-一般,1,26,0.038462,1,4,2.386294
2,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,._記号-一般,1,26,0.038462,1,4,2.386294
3,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,1_名詞-数,1,26,0.038462,1,4,2.386294
4,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd_名詞-固有名詞-一般,1,26,0.038462,1,4,2.386294


## TFIDF値を算出し新規カラムを作成

In [3]:
idf_merge_df['tf_idf'] = idf_merge_df['tf'] * idf_merge_df['idf']
idf_merge_df.head()

,内容,original_type,count,word_count,tf,word_include_text_count,all_text_count,idf,tf_idf
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,(_記号-一般,1,26,0.038462,1,4,2.386294,0.091781
1,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,)_記号-一般,1,26,0.038462,1,4,2.386294,0.091781
2,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,._記号-一般,1,26,0.038462,1,4,2.386294,0.091781
3,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,1_名詞-数,1,26,0.038462,1,4,2.386294,0.091781
4,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd_名詞-固有名詞-一般,1,26,0.038462,1,4,2.386294,0.091781


In [4]:
# 複数の文書に出現する単語（ここではParse_名詞-固有名詞-一般)を抽出し、各tf_idfの値を確認
idf_merge_df.query("original_type == 'Parse_名詞-固有名詞-一般' ")

,内容,original_type,count,word_count,tf,word_include_text_count,all_text_count,idf,tf_idf
5,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,Parse_名詞-固有名詞-一般,1,26,0.038462,2,4,1.693147,0.065121
6,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「＼n」で区切り、...,Parse_名詞-固有名詞-一般,1,30,0.033333,2,4,1.693147,0.056438


In [5]:
# pklファイルとして保存
idf_merge_df.to_pickle('tfidf_df.pkl')

## 単語毎の標準偏差を算出

In [6]:
std_df = idf_merge_df[['original_type', 'tf_idf']]
std_df = std_df.groupby('original_type').std(ddof=0)#標準偏差をn法で計算するには ddof=0
std_df = std_df.reset_index().rename(columns={'tf_idf':'std_tf_idf'})
std_df = std_df.sort_values(by='std_tf_idf', ascending=False)# 標準偏差が大きい＝特徴がある単語
std_df

,original_type,std_tf_idf
32,で_助詞-格助詞-一般,0.029276
67,＼_記号-一般,0.023088
20,「_記号-括弧開,0.023088
22,」_記号-括弧閉,0.019835
26,する_動詞-自立,0.019791
44,れる_動詞-接尾,0.018747
25,が_助詞-格助詞-一般,0.018747
14,t_名詞-固有名詞-組織,0.015643
51,単語_名詞-一般,0.015239
55,型_名詞-接尾-一般,0.015142


In [13]:
# pklファイルとして保存
std_df.to_pickle('std_df.pkl')

### （必要ないかもしれないが）TFIDF値とTFIDF値の標準偏差の値が入ったデータフレーム同士を結合

In [7]:
tfidf_df = idf_merge_df[['内容', 'original_type', 'tf_idf', 'count']]
tfidf_df.head()

,内容,original_type,tf_idf,count
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,(_記号-一般,0.091781,1
1,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,)_記号-一般,0.091781,1
2,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,._記号-一般,0.091781,1
3,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,1_名詞-数,0.091781,1
4,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd_名詞-固有名詞-一般,0.091781,1


In [8]:
len(tfidf_df)

105

In [10]:
std_tfidf_merge_df = pd.merge(tfidf_df, std_df, on='original_type', how='left')
std_tfidf_merge_df.shape

(105, 5)

In [12]:
std_tfidf_merge_df.head()

,内容,original_type,tf_idf,count,std_tf_idf
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,(_記号-一般,0.091781,1,0.0
1,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,)_記号-一般,0.091781,1,0.0
2,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,._記号-一般,0.091781,1,0.0
3,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,1_名詞-数,0.091781,1,0.0
4,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd_名詞-固有名詞-一般,0.091781,1,0.0


In [11]:
# マージした後に欠損値が発生したかどうか確認
std_tfidf_merge_df.isnull().any()

内容               False
original_type    False
tf_idf           False
count            False
std_tf_idf       False
dtype: bool